### Gene Correlation Matrix: Analyzing Expression Data in AnnData Format

This Jupyter Notebook demonstrates how to download and analyze gene expression data in AnnData format based on a specified tissue or cell type. It filters out genes with zero expression values and computes a correlation matrix for a specified gene.

#### Steps:

1. **Open and Query Data:**
   - Access gene expression data using `cellxgene_census.open_soma()`.
   - Query data for 'Homo sapiens' and filter by cell type ('adipocyte') or tissue type ('Adipose').

2. **Filter Genes:**
   - Set gene names (`adata.var['feature_id']`).
   - Filter out genes with zero expression values.

3. **Convert to DataFrame:**
   - Convert the filtered expression data into a Pandas DataFrame.

4. **Filter Samples:**
   - Remove samples where the gene of interest has zero expression.
   - Print the percentage of samples with non-zero expression for the gene of interest.

5. **Calculate Correlations:**
   - Compute Pearson correlation coefficients for the gene of interest.
   - Return and print the top 500 most positively and negatively correlated genes.

#### Usage Instructions:

- Modify the `cell_type`, `tissue_type`, and `gene_of_interest` variables as needed.
- Use `get_coexpression_matrix(gene, tissue, cell_type, k=500)` to obtain top correlated genes.

#### Notes:

- Ensure access to the appropriate AnnData formatted dataset.
- Experiment with different parameters to explore gene expression correlations.


### Downloading Dependencies

To run this notebook, ensure you have the necessary libraries installed:

- `cellxgene_census` for accessing gene expression data.
- `pandas` for data manipulation and analysis.
- `numpy` for numerical operations.
- `scipy.stats` for statistical calculations, including Pearson correlation (`pearsonr`).


In [2]:
%pip install cellxgene-census pandas numpy scipy

Note: you may need to restart the kernel to use updated packages.


### Importing Required Libraries

This code cell imports necessary libraries for data analysis:

- `cellxgene_census`: Imports functionality for working with cellxgene data.
- `pandas` (`pd`): Imports the Pandas library for data manipulation and analysis.
- `numpy` (`np`): Imports NumPy for numerical computing operations.
- `pearsonr` from `scipy.stats`: Imports the `pearsonr` function specifically for computing Pearson correlation coefficients.


In [3]:
import cellxgene_census
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

### Function to Get Co-Expression Matrix

The following code defines a function `get_coexpression_matrix` that performs the following steps:

1. **Open and Query Data:**
   - Uses `cellxgene_census.open_soma()` to access the gene expression data.
   - Queries data for the organism 'Homo sapiens' and filters by a specific cell type (`cell_type`) using `obs_value_filter=f"cell_type == '{cell_type}'"`.

2. **Ensure Correct Gene Names and Filter Genes:**
   - Checks and sets gene names (`adata.var['feature_id']`).
   - Filters out genes with zero expression values (`adata.X > 0`).

3. **Convert Data to DataFrame:**
   - Converts the filtered expression data into a Pandas DataFrame (`df_expression`).

4. **Filter Samples Based on Gene of Interest Expression:**
   - Filters out samples where the gene of interest has zero expression.
   - Calculates and prints the percentage of samples with non-zero expression for the gene of interest.

5. **Calculate Pearson Correlations:**
   - Computes Pearson correlation coefficients between the gene of interest and other genes in `df_expression`.
   - Sorts and returns the top 500 most positively and negatively correlated genes.

The function is then run with an example gene (`ENSG00000140718`) and the specified tissue and cell type (`Adipose` and `adipocyte`). The top 10 most positively and negatively correlated genes are printed.

#### Usage Instructions:

- Call `get_coexpression_matrix(gene, tissue, cell_type, k=500)` with your desired gene, tissue type, and cell type.
- The function will return two lists: the top 500 most positively and negatively correlated genes.
- Modify `k` to adjust the number of top correlations returned.

Example:
```python
top_positive, top_negative = get_coexpression_matrix('ENSG00000140718', 'Adipose', 'adipocyte', k=500)


In [9]:
def get_coexpression_matrix(gene, tissue, cell_type, k=500):
    with cellxgene_census.open_soma() as census:
        # Query the data for a specific organism and cell types
        adata = cellxgene_census.get_anndata(
            census=census,
            organism="Homo sapiens",
            obs_value_filter=f"cell_type == '{cell_type}'", # use obs_value_filter=f"tissue_general == '{tissue}'" if you want to filter with tissue type
            column_names={"obs": ["assay", "cell_type", "tissue", "tissue_general", "suspension_type", "disease"]},
        )

        # Ensure the gene names are set correctly
        if 'feature_id' in adata.var.columns:  # Adjust column name as needed
            adata.var_names = adata.var['feature_id']
        else:
            print("Gene names column 'feature_id' not found in var DataFrame")

        # Filter out genes with zero expression values
        gene_expression_sum = np.array((adata.X > 0).sum(axis=0)).flatten()
        adata_filtered = adata[:, gene_expression_sum > 0]

        # Convert the filtered expression data to a DataFrame
        df_expression = pd.DataFrame(adata_filtered.X.toarray(), columns=adata_filtered.var['feature_id'])

        # Check if the gene of interest is in the dataset
        if gene in df_expression.columns:
            # Filter out samples where the gene of interest is not expressed (expression value = 0)
            non_zero_samples = df_expression[df_expression[gene] > 0]
            
            # Calculate percentage of samples with non-zero expression for the gene of interest
            total_samples = df_expression.shape[0]
            non_zero_sample_count = non_zero_samples.shape[0]
            non_zero_percentage = (non_zero_sample_count / total_samples) * 100

            print(f"Total samples: {total_samples}")
            print(f"Samples with non-zero expression for '{gene}': {non_zero_sample_count} ({non_zero_percentage:.2f}%)")

            # Calculate Pearson correlation coefficients and p-values
            correlations = {}
            for g in non_zero_samples.columns:
                if g != gene:
                    corr, p_value = pearsonr(non_zero_samples[gene], non_zero_samples[g])
                    if p_value < 0.05:
                        correlations[g] = corr
                        
            # Sort correlations
            sorted_correlations = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
            top_positive = sorted_correlations[:k]
            top_negative = sorted_correlations[-k:]

            return top_positive, top_negative
        else:
            print(f"Gene of interest '{gene}' not found in the dataset.")
            return [], []

### Selecting Tissue or Cell Type and Gene of Interest

The below code cell sets variables to specify the tissue or cell type and a specific gene for analysis:


In [10]:
gene_of_interest = 'ENSG00000140718'
tissue_type = 'Adipose'
cell_type = 'adipocyte'

### Running the Co-Expression Matrix Function

The following code calls the `get_coexpression_matrix` function to obtain the top 500 most positively and negatively correlated genes for a specified gene of interest, tissue type, and cell type.


In [11]:
top_positive, top_negative = get_coexpression_matrix(gene_of_interest, tissue_type, cell_type, k=500)

The "stable" release is currently 2024-07-01. Specify 'census_version="2024-07-01"' in future calls to open_soma() to ensure data consistency.
/tmp/ipykernel_2985221/3158660545.py:4: FutureWarning: The argument `column_names` is deprecated and will be removed in a future release. Please use `obs_column_names` and `var_column_names` instead.
  adata = cellxgene_census.get_anndata(


Total samples: 81378
Samples with non-zero expression for 'ENSG00000140718': 53193 (65.37%)


/tmp/ipykernel_2985221/3158660545.py:41: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_value = pearsonr(non_zero_samples[gene], non_zero_samples[g])


### Printing Top Correlated Genes

The following code prints the top 10 most positively correlated genes with the specified gene of interest. Each gene is listed along with its Pearson correlation coefficient.

In [12]:
print("Top 10 most positively correlated genes:")
for gene, corr in top_positive[:10]:
    print(f"{gene}: correlation = {corr:.3f}")

Top 10 most positively correlated genes:
ENSG00000181722: correlation = 0.700
ENSG00000230590: correlation = 0.682
ENSG00000116117: correlation = 0.675
ENSG00000090905: correlation = 0.675
ENSG00000131558: correlation = 0.667
ENSG00000144357: correlation = 0.664
ENSG00000184903: correlation = 0.663
ENSG00000152818: correlation = 0.658
ENSG00000144036: correlation = 0.656
ENSG00000164330: correlation = 0.654


The following code prints the top 10 most negatively correlated genes with the specified gene of interest. Each gene is listed along with its Pearson correlation coefficient.

In [13]:
print("\nTop 10 most negatively correlated genes:")
for gene, corr in top_negative[:10]:
    print(f"{gene}: correlation = {corr:.3f}")


Top 10 most negatively correlated genes:
ENSG00000233932: correlation = 0.009
ENSG00000236301: correlation = 0.009
ENSG00000255307: correlation = 0.009
ENSG00000257366: correlation = 0.009
ENSG00000263435: correlation = 0.009
ENSG00000233411: correlation = 0.009
ENSG00000272788: correlation = 0.009
ENSG00000287076: correlation = 0.009
ENSG00000180697: correlation = 0.009
ENSG00000231815: correlation = 0.009
